In [98]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [99]:
if torch.cuda.is_available():
    device = "cuda"
elif torch.backends.mps.is_available():
    device = "mps"
else:
    device = "cpu"

print(f"Using {device} device!")

Using mps device!


In [100]:
# download training data from open datasets
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# download test data from open datasets
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

In [101]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

In [102]:
training_data

Dataset FashionMNIST
    Number of datapoints: 60000
    Root location: data
    Split: Train
    StandardTransform
Transform: ToTensor()

In [103]:
batch_size = 64
# create data loaders
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)
# get one batch of training data
train_features, train_labels = next(iter(train_dataloader))
print(f"Feature batch shape: {train_features.size()}")
print(f"Labels batch shape: {train_labels.size()}")

Feature batch shape: torch.Size([64, 1, 28, 28])
Labels batch shape: torch.Size([64])


In [104]:
# LeNet-5 model
# self.cnn = nn.Sequential(
#             nn.Conv2d(1, 6, kernel_size=5, padding=2),
#             nn.Sigmoid(),
#             nn.AvgPool2d(kernel_size=2, stride=2),
#             nn.Conv2d(6, 16, kernel_size=5, padding=0),
#             nn.AvgPool2d(kernel_size=2, stride=2),
#             nn.Flatten(),
#             nn.Linear(5 * 5 * 16, 120),
#             nn.Sigmoid(),
#             nn.Linear(120, 84),
#             nn.Sigmoid(),
#             nn.Linear(84, 10),
#         )

In [ ]:
class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.cnn = nn.Sequential(
            # convolutional layer
            nn.Conv2d(1, 10, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),

            # convolutional layer
            nn.Conv2d(10, 30, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),

            # flatten
            nn.Flatten(),

            # fully connected layer
            nn.Linear(7 * 7 * 30, 512),
            nn.ReLU(),
            nn.Dropout(),

            # fully connected layer
            nn.Linear(512, 128),
            nn.ReLU(),
            nn.Dropout(),

            # output layer
            nn.Linear(128, 10),
        )

    def forward(self, x):
        x = self.cnn(x)
        return x

In [106]:
model_cnn = CNN().to(device)
print(model_cnn)

CNN(
  (cnn): Sequential(
    (0): Conv2d(1, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(10, 30, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Flatten(start_dim=1, end_dim=-1)
    (7): Linear(in_features=1470, out_features=512, bias=True)
    (8): ReLU()
    (9): Dropout(p=0.5, inplace=False)
    (10): Linear(in_features=512, out_features=128, bias=True)
    (11): ReLU()
    (12): Dropout(p=0.5, inplace=False)
    (13): Linear(in_features=128, out_features=10, bias=True)
  )
)


In [107]:
# define loss function
loss_fn = nn.CrossEntropyLoss()
# define optimizer
optimizer = torch.optim.Adam(
    model_cnn.parameters(), 
    lr=0.001
)

In [110]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        # compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(
        f"Test Error: \n Accuracy: {(100 * correct):>0.1f}%, Avg loss: {test_loss:>8f} \n"
    )
epochs = 5
for t in range(epochs):
    print(f"Epoch {t + 1}\n-------------------------------")
    train_loop(train_dataloader, model_cnn, loss_fn, optimizer)
    test_loop(test_dataloader, model_cnn, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 0.147242  [    0/60000]
loss: 0.233396  [ 6400/60000]
loss: 0.122404  [12800/60000]
loss: 0.319275  [19200/60000]
loss: 0.228431  [25600/60000]
loss: 0.300780  [32000/60000]
loss: 0.257967  [38400/60000]
loss: 0.177851  [44800/60000]
loss: 0.217631  [51200/60000]
loss: 0.216368  [57600/60000]
Test Error: 
 Accuracy: 91.0%, Avg loss: 0.254331 

Epoch 2
-------------------------------
loss: 0.115250  [    0/60000]
loss: 0.228452  [ 6400/60000]
loss: 0.174608  [12800/60000]
loss: 0.217636  [19200/60000]
loss: 0.194922  [25600/60000]
loss: 0.324277  [32000/60000]
loss: 0.237678  [38400/60000]
loss: 0.284424  [44800/60000]
loss: 0.190237  [51200/60000]
loss: 0.215831  [57600/60000]
Test Error: 
 Accuracy: 91.5%, Avg loss: 0.248076 

Epoch 3
-------------------------------
loss: 0.146969  [    0/60000]
loss: 0.272756  [ 6400/60000]
loss: 0.175731  [12800/60000]
loss: 0.205047  [19200/60000]
loss: 0.244863  [25600/60000]
loss: 0.338328  [32000/600

In [111]:
# convert to TorchScript
model_scripted = torch.jit.script(model_cnn)

# write to disk
model_scripted.save("fashion.pt")